In [53]:
import json
import pickle
from sys_prompt import *

In [4]:
with open("./dev_20240627/dev_tied_append.json", "r") as f:
    ques_sql_lis = json.load(f)

with open("./dev_20240627/dev_tables.json", "r") as f:
    table_lis = json.load(f)

In [5]:
len(ques_sql_lis), len(table_lis)

(42, 11)

In [6]:
print(ques_sql_lis[0].keys())
print(table_lis[0].keys())

dict_keys(['question_id', 'db_id', 'question', 'evidence', 'SQL', 'difficulty'])
dict_keys(['db_id', 'table_names_original', 'table_names', 'column_names_original', 'column_names', 'column_types', 'primary_keys', 'foreign_keys'])


In [38]:
ques_sql_lis[0]["question"]

'What is the complete address of the school with the lowest excellence rate? Indicate the Street, City, Zip and State.'

In [7]:
table_lis[0]

{'db_id': 'debit_card_specializing',
 'table_names_original': ['customers',
  'gasstations',
  'products',
  'transactions_1k',
  'yearmonth'],
 'table_names': ['customers',
  'gas stations',
  'products',
  'transactions',
  'year and month'],
 'column_names_original': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'Segment'],
  [0, 'Currency'],
  [1, 'GasStationID'],
  [1, 'ChainID'],
  [1, 'Country'],
  [1, 'Segment'],
  [2, 'ProductID'],
  [2, 'Description'],
  [3, 'TransactionID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'CustomerID'],
  [3, 'CardID'],
  [3, 'GasStationID'],
  [3, 'ProductID'],
  [3, 'Amount'],
  [3, 'Price'],
  [4, 'CustomerID'],
  [4, 'Date'],
  [4, 'Consumption']],
 'column_names': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'client segment'],
  [0, 'Currency'],
  [1, 'Gas Station ID'],
  [1, 'Chain ID'],
  [1, 'Country'],
  [1, 'chain segment'],
  [2, 'Product ID'],
  [2, 'Description'],
  [3, 'Transaction ID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'Customer ID'],
  [3, 'Card I

In [35]:
def get_structure_schema(schema_data):
    db_id = schema_data.get("db_id")
    table_names_original = schema_data.get("table_names_original")
    table_names = schema_data.get("table_names")
    column_names_original = schema_data.get("column_names_original")
    column_names = schema_data.get("column_names")
    column_types = schema_data.get("column_types")
    
    structured_schema = {"db_id": db_id, "tables": {}}
    
    # Iterate over each table
    for i, table_name in enumerate(table_names_original):
        table_alias = table_names[i]
        columns = []
        
        # For each column in this table, match original names with alias and types
        for col_info in zip(column_names_original, column_names, column_types):
            original_col_info, alias_col_info, col_type = col_info
            if original_col_info[0] == i:  # Check if this column belongs to the current table
                columns.append({
                    "original_name": original_col_info[1],
                    "alias_name": alias_col_info[1],
                    "type": col_type
                })
        
        # Store the columns for each table
        structured_schema["tables"][table_alias] = {"columns": columns}
    
    return structured_schema

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}

In [49]:
finetune_dataset = []

for i in ques_sql_lis:
    data_pt = {"messages": [], 
               }
    # user_pt = {"system": }
    db_id = i["db_id"]
    sql = i["SQL"]
    ques = i["question"]
    ques_id = i["question_id"]


    for j in table_lis:
        j["db_id"] == db_id
        struct_db = get_structure_schema(j)
        sys_dict = {"role": "system", "content": sys_prompt.format(database_schema = struct_db)}
        user_dict = {"role": "user", "content": ques}
        asst_dict = {"role": "assistant", "content": sql}

    data_pt["messages"].append([sys_dict, user_dict, asst_dict])

    finetune_dataset.append(data_pt)
        
# print(finetune_dataset)
        

In [51]:
len(finetune_dataset)

42

In [52]:
finetune_dataset

[{'messages': [[{'role': 'system',
     'content': "You are an AI assistant trained to convert natural language queries into SQL queries. When a user provides a question in English, you will generate the corresponding SQL query based on the provided database schema.\n\nYour task is to:\n\nUnderstand the user’s query in natural language.\nGenerate a SQL query that answers the question based on the provided database schema.\nEnsure the SQL query is syntactically correct, adheres to the database schema, and returns the correct results.\nDo not assume any context beyond the database schema provided.\nIn your query, only use the actual table names and not the alias (they are only for your reference and might be used by the user)\ndatabase schema: {'db_id': 'codebase_community', 'tables': {'badges': {'columns': [{'original_name': 'Id', 'alias_name': 'Id', 'type': 'integer'}, {'original_name': 'UserId', 'alias_name': 'User Id', 'type': 'integer'}, {'original_name': 'Name', 'alias_name': 'Name

In [48]:
finetune_dataset[0]["messages"][0][0]["content"]

"You are an AI assistant trained to convert natural language queries into SQL queries. When a user provides a question in English, you will generate the corresponding SQL query based on the provided database schema.\n\nYour task is to:\n\nUnderstand the user’s query in natural language.\nGenerate a SQL query that answers the question based on the provided database schema.\nEnsure the SQL query is syntactically correct, adheres to the database schema, and returns the correct results.\nDo not assume any context beyond the database schema provided.\nIn your query, only use the actual table names and not the alias (they are only for your reference and might be used by the user)\ndatabase schema: {'db_id': 'codebase_community', 'tables': {'badges': {'columns': [{'original_name': 'Id', 'alias_name': 'Id', 'type': 'integer'}, {'original_name': 'UserId', 'alias_name': 'User Id', 'type': 'integer'}, {'original_name': 'Name', 'alias_name': 'Name', 'type': 'text'}, {'original_name': 'Date', 'alia

In [ ]:


# Sample schema data (replace with your actual input)
schema_data = {
    "db_id": "debit_card_specializing",
    "table_names_original": ['customers', 'gasstations', 'products', 'transactions_1k', 'yearmonth'],
    "table_names": ['customers', 'gas stations', 'products', 'transactions', 'year and month'],
    "column_names_original": [
        [-1, '*'], 
        [0, 'CustomerID'], [0, 'Segment'], [0, 'Currency'],
        [1, 'GasStationID'], [1, 'ChainID'], [1, 'Country'], [1, 'Segment'],
        [2, 'ProductID'], [2, 'Description'],
        [3, 'TransactionID'], [3, 'Date'], [3, 'Time'], [3, 'CustomerID'], [3, 'CardID'],
        [3, 'GasStationID'], [3, 'ProductID'], [3, 'Amount'], [3, 'Price'],
        [4, 'CustomerID'], [4, 'Date'], [4, 'Consumption']
    ],
    "column_names": [
        [-1, '*'],
        [0, 'CustomerID'], [0, 'client segment'], [0, 'Currency'],
        [1, 'Gas Station ID'], [1, 'Chain ID'], [1, 'Country'], [1, 'chain segment'],
        [2, 'Product ID'], [2, 'Description'],
        [3, 'Transaction ID'], [3, 'Date'], [3, 'Time'], [3, 'Customer ID'], [3, 'Card ID'],
        [3, 'Gas Station ID'], [3, 'Product ID'], [3, 'Amount'], [3, 'Price'],
        [4, 'Customer ID'], [4, 'Date'], [4, 'Consumption']
    ],
    "column_types": [
        "text", "integer", "text", "text", "integer", "integer", "text", "text",
        "integer", "text", "integer", "date", "text", "integer", "integer", "integer",
        "integer", "integer", "real", "integer", "text", "real"
    ]
}

# Get the structured schema
structured_schema = structure_schema(schema_data)

# Print the structured schema
import json
print(json.dumps(structured_schema, indent=4))


{
    "database_id": "debit_card_specializing",
    "tables": {
        "customers": {
            "columns": [
                {
                    "original_name": "CustomerID",
                    "alias_name": "CustomerID",
                    "type": "integer"
                },
                {
                    "original_name": "Segment",
                    "alias_name": "client segment",
                    "type": "text"
                },
                {
                    "original_name": "Currency",
                    "alias_name": "Currency",
                    "type": "text"
                }
            ]
        },
        "gas stations": {
            "columns": [
                {
                    "original_name": "GasStationID",
                    "alias_name": "Gas Station ID",
                    "type": "integer"
                },
                {
                    "original_name": "ChainID",
                    "alias_name": "Chain ID",
        

In [57]:
with open("test", "wb") as fp:
    pickle.dump(finetune_dataset, fp)
with open("finetune_dataset", "rb") as fp:
    b = pickle.load(fp)

In [58]:
b == finetune_dataset

True

In [32]:
structure_schema(input_data)

{'database_id': 'debit_card_specializing',
 'tables': {'customers': {'columns': [{'original_name': 'CustomerID',
     'alias_name': 'CustomerID',
     'type': 'integer'},
    {'original_name': 'client_segment',
     'alias_name': 'client segment',
     'type': 'text'},
    {'original_name': 'Currency', 'alias_name': 'Currency', 'type': 'text'}]},
  'gas stations': {'columns': [{'original_name': 'Gas_Station_ID',
     'alias_name': 'Gas Station ID',
     'type': 'integer'},
    {'original_name': 'Chain_ID', 'alias_name': 'Chain ID', 'type': 'integer'},
    {'original_name': 'Country', 'alias_name': 'Country', 'type': 'text'},
    {'original_name': 'chain_segment',
     'alias_name': 'chain segment',
     'type': 'text'}]},
  'products': {'columns': [{'original_name': 'Product_ID',
     'alias_name': 'Product ID',
     'type': 'integer'},
    {'original_name': 'Description',
     'alias_name': 'Description',
     'type': 'text'}]},
  'transactions': {'columns': [{'original_name': 'Transa

In [ ]:
for i in ques_sql_lis:
    db_id = i["db_id"]
    tab_nm = i["table_names_original"]
    col